In [1]:
import numpy as np
from scipy.linalg import expm, logm
import sys
import copy
import time
import cv2
import os
from PIL import Image

In [2]:
################ Pre-defined parameters and functions below (can change if needed) ################
PI = np.pi
# 20Hz
SPIN_RATE = 20  

# UR3 home location
home = [270*PI/180.0, -90*PI/180.0, 90*PI/180.0, -90*PI/180.0, -90*PI/180.0, 135*PI/180.0]  

# UR3 current position, using home position for initialization
current_position = copy.deepcopy(home)  

thetas = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

digital_in_0 = 0
analog_in_0 = 0.0

suction_on = True
suction_off = False

current_io_0 = False
current_position_set = False

In [3]:
from matplotlib.pyplot import imshow
from matplotlib.pyplot import cm
def find_keypoints(image):
    """Gets keypoints from the given image
    Parameters
    ----------
    image : np.ndarray
        The given image (before or after preprocessing)
    Returns
    -------
    keypoints
        a list of keypoints detected in image coordinates
    """
    keypoints = []
    return keypoints


# Create directory if it doesn't exist
if not os.path.exists("threshold_images"):
    os.makedirs("threshold_images")

In [4]:
def IMG2W(row, col, image):
    """Transform image coordinates to world coordinates

    Parameters
    ----------
    row : int
        Pixel row position
    col : int
        Pixel column position
    image : np.ndarray
        The given image (before or after preprocessing)

    Returns
    -------
    x : float
        x position in the world frame
    y : float
        y position in the world frame
    """
    x, y = 0.0, 0.0
    return x, y

In [5]:
def draw_image(world_keypoints):
    """Draw the image based on detecte keypoints in world coordinates

    Parameters
    ----------
    world_keypoints:
        a list of keypoints detected in world coordinates
    """
    pass


"""
Program run from here
"""

'\nProgram run from here\n'

In [111]:
def find_keypoints(image):
    """Gets keypoints along center of lines with reduced density
    
    Parameters
    ----------
    image : np.ndarray
        The given image (before or after preprocessing)
    """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    # Threshold to ensure clean binary image
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    
    # Fill in the lines to get solid regions
    kernel = np.ones((5,5), np.uint8)
    filled = cv2.dilate(binary, kernel, iterations=1)
    
    # Get the distance transform
    dist = cv2.distanceTransform(filled, cv2.DIST_L2, 5)
    
    # Threshold the distance transform to get center line
    _, center = cv2.threshold(dist, 1, 255, cv2.THRESH_BINARY)
    center = center.astype(np.uint8)
    
    # Find contours of the center line
    contours, _ = cv2.findContours(center, cv2.RETR_LIST, 
                                 cv2.CHAIN_APPROX_SIMPLE)
    
    # Process contours to get keypoints
    keypoints = []
    min_length = 10  # Increased minimum length
    spacing = 20     # Increased spacing between points
    
    for contour in contours:
        if len(contour) < min_length:
            continue
            
        # Use contour approximation to reduce points
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, False)
        
        # Sample points with larger spacing
        for i in range(0, len(approx), spacing):
            point = approx[i][0]
            keypoints.append(tuple(point))
            
    # Create debug images
    debug_image = image.copy()
    if len(debug_image.shape) == 2:
        debug_image = cv2.cvtColor(debug_image, cv2.COLOR_GRAY2BGR)
    
    # Draw keypoints
    keypoint_image = debug_image.copy()
    for x, y in keypoints:
        cv2.circle(keypoint_image, (x, y), 3, (0, 0, 255), -1)
        
    # Save debug images
    cv2.imwrite('debug_1_binary.png', binary)
    cv2.imwrite('debug_2_filled.png', filled)
    cv2.imwrite('debug_3_center.png', center)
    cv2.imwrite('debug_4_keypoints.png', keypoint_image)
    
    return keypoints

In [101]:
zigzag = cv2.imread("images/zigzag.jpg")
turkey = cv2.imread("images/turkey.png")
status = cv2.imread("images/status.png")


In [ ]:
def find_keypoints(image):
    """Gets keypoints from image following lab manual guidelines
    
    Parameters
    ----------
    image : np.ndarray
        The given image (before or after preprocessing)
    
    Returns
    -------
    keypoints : list
        A list of (x,y) tuples representing detected keypoints
    """
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
    
    edges = cv2.Canny(binary, 50, 150, apertureSize=3)
    
    lines = cv2.HoughLinesP(
        edges,
        rho=1,
        theta=np.pi/180,
        threshold=20,
        minLineLength=20,
        maxLineGap=10
    )
    
    # 5. Convert detected lines to keypoints
    keypoints = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            
            # Add endpoints
            keypoints.append((x1, y1))
            keypoints.append((x2, y2))
            
            # Add intermediate points for longer lines
            length = np.sqrt((x2-x1)**2 + (y2-y1)**2)
            if length > 30:
                # Add a point in the middle
                mx = (x1 + x2) // 2
                my = (y1 + y2) // 2
                keypoints.append((mx, my))
    
    # Remove duplicates while maintaining order
    keypoints = list(dict.fromkeys(keypoints))
    
    # Create debug images to help with tuning
    debug_image = image.copy()
    if len(debug_image.shape) == 2:
        debug_image = cv2.cvtColor(debug_image, cv2.COLOR_GRAY2BGR)
        
    # Draw processing steps
    cv2.imwrite('debug_1_gray.png', gray)
    cv2.imwrite('debug_2_binary.png', binary)
    cv2.imwrite('debug_3_edges.png', edges)
    
    # Draw lines and keypoints
    line_image = debug_image.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.imwrite('debug_4_lines.png', line_image)
    
    keypoint_image = debug_image.copy()
    for x, y in keypoints:
        cv2.circle(keypoint_image, (x, y), 3, (0, 0, 255), -1)
    cv2.imwrite('debug_5_keypoints.png', keypoint_image)
    
    return keypoints

def IMG2W(row, col, image):
    """Transform image coordinates to world coordinates
    
    Parameters
    ----------
    row : int
        Pixel row position
    col : int
        Pixel column position
    image : np.ndarray
        The given image
        
    Returns
    -------
    x : float
        x position in world frame
    y : float
        y position in world frame
    """
    # Get image dimensions
    height, width = image.shape[:2]
    
    # Define the drawing area size in world coordinates (in meters)
    world_width = 0.2   # 20cm
    world_height = 0.2  # 20cm
    
    # Calculate scaling factors
    scale_x = world_width / width
    scale_y = world_height / height
    
    # Convert to world coordinates
    # Assuming (0.2, 0.2) is the bottom-left corner of drawing area
    x = 0.2 + (col * scale_x)
    y = 0.2 + ((height - row) * scale_y)  # Invert y-axis
    
    return x, y

In [112]:
# find_keypoints(status)
find_keypoints(zigzag)
find_keypoints(turkey)

[(np.int32(451), np.int32(744)),
 (np.int32(621), np.int32(739)),
 (np.int32(389), np.int32(736)),
 (np.int32(533), np.int32(713)),
 (np.int32(797), np.int32(707)),
 (np.int32(744), np.int32(706)),
 (np.int32(257), np.int32(700)),
 (np.int32(1022), np.int32(700)),
 (np.int32(736), np.int32(696)),
 (np.int32(244), np.int32(696)),
 (np.int32(216), np.int32(688)),
 (np.int32(751), np.int32(684)),
 (np.int32(774), np.int32(682)),
 (np.int32(1022), np.int32(674)),
 (np.int32(872), np.int32(672)),
 (np.int32(156), np.int32(668)),
 (np.int32(92), np.int32(668)),
 (np.int32(748), np.int32(667)),
 (np.int32(102), np.int32(660)),
 (np.int32(635), np.int32(658)),
 (np.int32(736), np.int32(654)),
 (np.int32(812), np.int32(646)),
 (np.int32(391), np.int32(642)),
 (np.int32(100), np.int32(624)),
 (np.int32(686), np.int32(619)),
 (np.int32(658), np.int32(611)),
 (np.int32(890), np.int32(603)),
 (np.int32(815), np.int32(592)),
 (np.int32(866), np.int32(589)),
 (np.int32(653), np.int32(582)),
 (np.int3

In [ ]:
# def find_keypoints(image):
#     """Gets keypoints along the centerlines of the black lines in the drawing
    
#     Parameters
#     ----------
#     image : np.ndarray
#         The given image (before or after preprocessing)
#     """
#     # Convert to grayscale if needed
#     if len(image.shape) == 3:
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     else:
#         gray = image.copy()
    
#     # Threshold to get black lines
#     _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    
#     # Create a copy for visualization
#     vis_image = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
    
#     # Get distance transform
#     dist = cv2.distanceTransform(binary, cv2.DIST_L2, 5)
    
#     # Threshold distance transform to get ridges (centerlines)
#     _, ridges = cv2.threshold(dist, 1, 255, cv2.THRESH_BINARY)
#     ridges = ridges.astype(np.uint8)
    
#     # Find contours of the ridges
#     contours, _ = cv2.findContours(ridges, cv2.RETR_LIST, 
#                                  cv2.CHAIN_APPROX_NONE)
    
#     keypoints = []
#     min_contour_length = 30
    
#     for contour in contours:
#         if len(contour) < min_contour_length:
#             continue
            
#         # Approximate the contour to get main points
#         epsilon = 0.01 * cv2.arcLength(contour, True)
#         approx = cv2.approxPolyDP(contour, epsilon, False)
        
#         # Get points with consistent spacing
#         for i in range(len(approx) - 1):
#             p1 = tuple(approx[i][0])
#             p2 = tuple(approx[i + 1][0])
            
#             # Calculate distance between points
#             dx = p2[0] - p1[0]
#             dy = p2[1] - p1[1]
#             dist = np.sqrt(dx*dx + dy*dy)
            
#             # Add points with fixed spacing
#             if dist > 20:
#                 steps = int(dist / 20)
#                 for j in range(steps + 1):
#                     t = j / steps
#                     x = int(p1[0] + dx * t)
#                     y = int(p1[1] + dy * t)
#                     keypoints.append((x, y))
#             else:
#                 keypoints.append(p1)
                
#         # Add last point
#         if len(approx) > 0:
#             keypoints.append(tuple(approx[-1][0]))
    
#     # Remove duplicates while maintaining order
#     keypoints = list(dict.fromkeys(keypoints))
    
#     # Draw debug visualizations
#     debug_image = image.copy()
#     if len(debug_image.shape) == 2:
#         debug_image = cv2.cvtColor(debug_image, cv2.COLOR_GRAY2BGR)
    
#     # Draw ridges
#     ridge_image = debug_image.copy()
#     cv2.drawContours(ridge_image, contours, -1, (0, 255, 0), 1)
    
#     # Draw keypoints and connecting lines
#     keypoint_image = debug_image.copy()
    
#     # Draw lines between consecutive points
#     for i in range(len(keypoints) - 1):
#         pt1 = keypoints[i]
#         pt2 = keypoints[i + 1]
#         cv2.line(keypoint_image, pt1, pt2, (0, 255, 0), 1)
    
#     # Draw keypoints
#     for x, y in keypoints:
#         cv2.circle(keypoint_image, (x, y), 2, (0, 0, 255), -1)
    
#     # Save debug images
#     cv2.imwrite('debug_1_binary.png', binary)
#     cv2.imwrite('debug_2_ridges.png', ridges)
#     cv2.imwrite('debug_3_centerlines.png', ridge_image)
#     cv2.imwrite('debug_4_keypoints.png', keypoint_image)
    
#     return keypoints